[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/e_iter_methods/iter_methods.ipynb)

In [ ]:
# Necessity libraries
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from math import *
import sys

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Iterative methods

## Steepest descent

The code implements the steepest descent algorithm using
$$
A=\begin{pmatrix} 3 & 0.8 \\ 0.8 & 1.2 \end{pmatrix}, \qquad f= \begin{pmatrix} 4 \\ 6 \end{pmatrix}.
$$
This has solution
$$
u_* = \begin{pmatrix} 0 \\ 5 \end{pmatrix}.
$$
The program reaches a tolerance of \(10^{-10}\) in 43 iterations. It takes a zig-zag path to reach the solution.

### Set up

In [ ]:
# Matrix and vector
A = np.array([[3, 0.8], [0.8, 1.2]])
# A=np.array([[2,0],[0,2]])       # Alternative best case: circle
f = np.array([4, 6])

# Initial guess and tolerance level
u = np.array([0, 0])
eps = 1e-10

# Storage for solutions
max_iters = 100
u_k = np.empty((100, 2))
u_k[0, :] = u

### Improved steepest descent algorithm

In [ ]:
# Steepest descent algorithm
r = f-np.dot(A, u)
k = 1
while True:

    # Steepest descent algorithm steps
    w = np.dot(A, r)
    a = np.dot(r, r)/np.dot(r.T, w)
    u = u+a*r
    r = r-a*w

    # Store current solution
    u_k[k, :] = u

    # Check for too many iterations
    if k > 100:
        print("Too many iterations")
        sys.exit()

    # Check for convergence
    k += 1
    if np.linalg.norm(r) < eps:
        break

### Visualize

In [ ]:
# Plot results - create contours of phi function
n = 100
xx = np.linspace(-4, 6, n)
yy = np.linspace(-2, 8, n)
X, Y = np.meshgrid(xx, yy)
pxy = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        u = np.array([X[i, j], Y[i, j]])
        pxy[i, j] = 0.5*np.dot(u.T, np.dot(A, u))-np.dot(u, f)

fig, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=300)
# Plot contours
contourf = ax.contourf(X, Y, pxy, 16, alpha=.75)
contour = ax.contour(X, Y, pxy, 16, colors='black')
# Plot results: overlay progress of algorithm
ax.plot(u_k[:k, 0], u_k[:k, 1], color='red', marker='x')

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
plt.show()

## Simple conjugate gradient for 1D Poisson

The code uses CG to solve the one-dimensional Poisson equation for $u(x)$,
$$
\frac{\partial^2 u}{\partial x^2} = f
$$
on the interval $[0,1]$, with Dirichlet conditions $u(0)=u(1)=0$.

### Set up

In [ ]:
# Total number of grid points, and mesh spacing
n = 9
h = 1/(n+1)
hfac = 1/(h*h)

# Calculates the multiplication Au for a given input vector u, without
# explicitly building the matrix A
def mul_A(u):
    Au = -2*u

    Au[0] += u[1]
    for i in range(1, n-1):
        Au[i] += u[i-1]+u[i+1]
    Au[n-1] += u[n-2]
    Au *= hfac
    return Au

### CG algorithm

In [ ]:
# Conjugate gradient algorithm
r_k = np.ones((n))
u_k = np.zeros((n))
p_k = np.copy(r_k)
rr = np.dot(r_k, r_k)
print("# 0", rr)
k = 1
# Store all u_k iterates
u_k_list = [u_k.copy()]

while k <= n:

    # Update solution by moving in the direction of the p vector
    w_k = mul_A(p_k)
    alpha = rr/np.dot(p_k, w_k)
    u_k += alpha*p_k
    r_k -= alpha*w_k

    # Store current solution
    u_k_list.append(u_k.copy())

    # Compute the 2-norm squared of the residual vector and check
    # for convergence
    new_rr = np.dot(r_k, r_k)
    print("#", k, new_rr)
    if new_rr < 1e-20:
        break

    # Compute new conjugate direction p
    beta_k = new_rr/rr
    rr = new_rr
    p_k = r_k+beta_k*p_k
    k += 1

# Print out the solution
print("\n", 0, 0)
for i in range(0, n):
    print(h*(i+1), u_k[i])
print(1, 0)

### Visualize

In [ ]:
# Plot u_k at each iteration
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

for i in range(len(u_k_list)):
    ax.plot(np.linspace(0, 1, n),
            u_k_list[i], '-o', color=plt.cm.viridis(i / len(u_k_list)), alpha=0.5, label=f'$u_{i}$')

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$u_k$')
ax.legend()
plt.show()

## Conjugate gradient for 2D Poisson

For $\epsilon = 10^{-4}$, with the program
\clink{e_iter_methods}{poisson_cg.py}{poisson\_\,cg.py}, we obtain the following
convergence results

The code uses CG to solve the two-dimensional Poisson equation for $u(x)$,
$$
\nabla^2 u = 1
$$
on the interval $[0,1]$.

### Set up

In [ ]:
# Grid size and grid spacing
n = 51
h = 1/(n-1)

# Generate the centered difference differentiation matrix for the Laplacian
de = 1/(h*h)
nn = n-2
cen = np.ones(nn*nn)*(4*de)
hor = np.ones(nn*nn-1)*(-de)
ver = np.ones(nn*(nn-1))*(-de)
for i in range(nn-1):
    hor[(i+1)*nn-1] = 0
A = sp.diags([cen, hor, hor, ver, ver], [0, 1, -1, nn, -nn])

# Right hand side: a vector of ones
rhs = np.ones(nn*nn)
norm_rhs = np.linalg.norm(rhs)

### CG algorithm

In [ ]:
# Solve for solution using the conjugate gradient method
TOL = 1.e-4
u_k = np.zeros(nn*nn)
r_k = rhs
p_k = r_k
count = 1

while True:

    # First half of conjugate gradient iteration
    w_k = A*p_k
    alpha_k = np.dot(r_k.T, r_k)/np.dot(p_k.T, w_k)
    u_k = u_k+alpha_k*p_k
    r_k_old = r_k
    r_k = r_k-alpha_k*w_k

    # Compute relative residual and use to it to check for convergence
    rel_residual = np.linalg.norm(r_k)/norm_rhs
    print("iteration=%d, relative residual=%g" % (count, rel_residual))
    if rel_residual < TOL:
        break

    # Second half of conjugate gradient iteration
    beta_k = np.dot(r_k.T, r_k)/np.dot(r_k_old.T, r_k_old)
    p_k = r_k+beta_k*p_k
    count += 1

# Print grid spacing and condition number. This may get very expensive for a
# large grid, since it uses a dense linear algebra routine to compute the
# condition number.
print("h=%g, condition number=%g\n" % (h, np.linalg.cond(A.todense())))

### Visualize

In [ ]:
# %matplotlib ipympl

# Assemble solution in a grid
uu=np.zeros((n,n))
for i in range(nn):
    for j in range(nn):
        uu[i+1,j+1]=u_k[i+nn*j]

xa=np.linspace(0,1,n)
mgx,mgy=np.meshgrid(xa,xa);

# Plot the solution using subplots style
fig, ax = plt.subplots(figsize=(8, 6), dpi=300, subplot_kw={"projection": "3d"})
surf = ax.plot_surface(mgx, mgy, uu, rstride=1, cstride=1, cmap=plt.cm.PuOr_r)

# Formatting
fig.colorbar(surf, shrink=0.4, aspect=10, pad=0.075)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
plt.show()